In [1]:
import numpy as np
from pyscf import gto, scf, fci

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for spinorbitals
def ONERDMFT_hartree_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals
def ONERDMFT_Umrigar_hartree_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            if not(a == b):
                for mu in range(0,M):
                    for nu in range(0,M):
                        for kappa in range(0,M):
                            for lamda in range(0,M):
                                energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [5]:
# for spinorbitals, I can only simulate orthogonality by excluding \alpha \beta, \beta \alpha from the sum since I know the first set of M are alpha and the second set of M are beta
def ONERDMFT_Umrigar_exchange_corelation_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M//2):
        for b in range(0,M//2):
            if not(a == b):
                for mu in range(0,M):
                    for nu in range(0,M):
                        for kappa in range(0,M):
                            for lamda in range(0,M):
                                energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  range(M//2,M):
        for b in range(M//2,M):
            if not(a == b):
                for mu in range(0,M):
                    for nu in range(0,M):
                        for kappa in range(0,M):
                            for lamda in range(0,M):
                                energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [6]:
# for spinorbitals, I can only simulate orthogonality by excluding \alpha \beta, \beta \alpha from the sum since I know the first set of M are alpha and the second set of M are beta
def ONERDMFT_exchange_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  range(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [7]:
mol = gto.Mole()
mol.atom = """
    Be    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" 
#mol.basis = "ccpvdz"
#mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

print(eri.shape)

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

C = mf.mo_coeff
h = mf.get_hcore()

N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
occu, naturalC = np.linalg.eigh(gamma)

(9, 9, 9, 9)
converged SCF energy = -14.5667640335096


In [8]:
# need to first make gamma and then expand into blockdiagonal so "fake" alpha and "beta" are seperated in this fashion
CE = expand_matrix(C)
gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
occu_aa, naturalC_aa = np.linalg.eigh(gamma)
occu = np.append(occu_aa, occu_aa)
naturalCE = expand_matrix(naturalC_aa)
naturalCTT = np.matmul(CE,naturalCE)

In [25]:
for i, n  in enumerate(occu):
    if n < 0:
        occu[i] = 0 
print(occu)

[0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.]


In [23]:
#print(naturalCTT)
E_H = ONERDMFT_hartree_energy(eri, naturalCTT, occu)
E_U = ONERDMFT_Umrigar_hartree_energy(eri, naturalCTT, occu)
E_x = ONERDMFT_exchange_energy(eri, naturalCTT, occu)
E_xc = ONERDMFT_Umrigar_exchange_corelation_energy(eri, naturalCTT, occu)
print(f"Based on expanded HF density matrix:\n Hartree Energy: {E_H}")
print(f"Exchange Energy: {E_x}\n Hartree + Exchange {E_H+E_x}")
print(f" Umrigar Hartree: {E_U}\n Umrigar Exchangecorrelation: {E_xc}\n  Umrigar Hartree + Exchangecorrelation: {E_U + E_xc}")

Based on expanded HF density matrix:
 Hartree Energy: 7.23294633837142
Exchange Energy: -2.6825540878306247
 Hartree + Exchange 4.550392250540796 
 Umrigar
Hartree: 4.698658560895182
 Umrigar Exchangecorrelation: -0.14826631035439877
  Umrigar Hartree + Exchangecorrelation: 4.550392250540783


In [11]:
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
FCIgamma = fci.direct_spin1.make_rdm1(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIgamma = FCIgamma/2.0
FCIoccu, FCInaturalC = np.linalg.eigh(FCIgamma)
FCInaturalCTT = np.matmul(C,FCInaturalC)
FCInaturalCTTE = expand_matrix(FCInaturalCTT)
FCIoccuE = np.append(FCIoccu,FCIoccu)
#print(FCInaturalCTTE)

In [24]:
print(FCIoccu)

[2.19023303e-05 2.19030313e-05 2.19037004e-05 1.01393176e-03
 3.28327641e-02 3.28328321e-02 3.28329282e-02 9.00486971e-01
 9.99934864e-01]


In [12]:
print(f"E_FCI {e} E_HF {mf.e_tot} E_c {e-mf.e_tot}")

E_FCI -14.613545269594093 E_HF -14.566764033509628 E_c -0.04678123608446505


In [13]:
E_H = ONERDMFT_hartree_energy(eri, FCInaturalCTTE, FCIoccuE)
E_U = ONERDMFT_Umrigar_hartree_energy(eri, FCInaturalCTTE, FCIoccuE)
E_x = ONERDMFT_exchange_energy(eri, FCInaturalCTTE, FCIoccuE)
E_xc = ONERDMFT_Umrigar_exchange_corelation_energy(eri, FCInaturalCTTE, FCIoccuE)
print(f"Based on expanded FCI density matrix:\n Hartree Energy: {E_H}\n Exchange Energy:{E_x}\n Umrigar Hartree: {E_U}\n Umrigar Exchangecorrelation: {E_xc}")

Based on expanded FCI density matrix:
 Hartree Energy: 7.267142831697578
 Exchange Energy:-2.6330007671336277
 Umrigar Hartree: 4.694238455285422
 Umrigar Exchangecorrelation: -0.14907389348298758


In [22]:
h = mf.get_hcore()
2*np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T))))+E_U+E_xc

-14.572862309624027

In [21]:
print(np.trace(np.matmul(h,P)))
print(np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma,C.T)))))

-9.558578142025207
-9.559013435713231
